# Integrating a User-provided simulator in the end-to-end AutoEmulate workflow 

### In this workflow we demonstrate the integration of a Cardiovascular simulator, Naghavi Model from ModularCirc in the end-to-end AutoEmulate workflow. 
Naghavi model is a 0D (zero-dimensional) computational model of the cardiovascular system, which is used to simulate blood flow and pressure dynamics in the heart and blood vessels.

This demo includes:
- Setting up parameter ranges 
- Creating sapmles
- Running the simulator to generate training data for the emulator 
- Using Autoemulate to find the best pre-processing technique and model tailored to the simulation data 
- Applying history matching to refine the model and enhance parameter ranges 
- Sensitvity Analysis 

## Additional dependency requirements

#### In this demonstration we are using the Naghavi Model Simulator from ModularCirc library. Therefore, the user needs to install the ModularCirc library in their existing AutoEmulate virtual environemnt as an additional dependency. 

In [22]:
# ! pip install git+https://github.com/alan-turing-institute/ModularCirc.git@dev

#### Create a dictionary called `parameters_range` which contains teh name of the simulator input parameters and their range.

In [ ]:
from autoemulate.simulations.naghavi_cardiac_ModularCirc import extract_parameter_ranges
# Usage example:
parameters_range = extract_parameter_ranges('/Users/mfamili/work/ModularCirc/Tutorials/Tutorial_03/Parameters_01.json')
parameters_range

dict

#### Use  `LatinHypercube` method from AutoEmulate to generate initial samples using the parameters range 

In [ ]:
import pandas as pd
import numpy as np
from autoemulate.experimental_design import LatinHypercube

# Generate Latin Hypercube samples
N_samples = 150
lhd = LatinHypercube(list(parameters_range.values()))
sample_array = lhd.sample(N_samples)
sample_df = pd.DataFrame(sample_array, columns=parameters_range.keys())

print("Number of parameters:", sample_df.shape[1], "Number of samples from each parameter:", sample_df.shape[0])
sample_df.head()

Number of parameters: 16 Number of samples from each parameter: 150


,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,la.v_ref,la.k_pas,lv.E_pas,lv.E_act,lv.v_ref,lv.k_pas
0,176.819881,0.406248,1049.271979,4.450300,6.502936,153.452000,4.532071,2.411829,0.652911,0.464574,14.813956,0.034605,1.324480,3.962469,6.865957,0.033686
1,232.861667,0.225987,1332.000265,3.842844,7.473219,111.762093,3.624132,2.388363,0.265285,0.570591,13.350931,0.019960,1.391763,2.089688,6.772118,0.013968
2,259.115820,0.357755,754.951511,2.467084,7.750360,77.374964,4.193461,5.858118,0.498185,0.598356,5.967148,0.029932,1.066198,3.248931,8.610645,0.011033
3,290.388461,0.171399,1060.303364,2.432098,4.634911,143.860543,5.299102,3.955810,0.510764,0.415178,7.590902,0.063273,0.947786,2.941678,5.492776,0.038869
4,359.713091,0.317270,867.335829,3.404646,12.311142,92.489421,6.093638,6.115941,0.357064,0.611183,8.315153,0.061478,0.692467,2.998555,12.413691,0.019499


## Wrapping your Simulator in AutoEmulate Simulator Base Class

#### 

In [ ]:
from autoemulate.simulations.naghavi_cardiac_ModularCirc import NaghaviSimulator
# Initialize simulator with specific outputs
simulator = NaghaviSimulator(
    parameters_range=parameters_range, 
    output_variables=['lv.P_i', 'lv.P_o'],  # Only the ones you're interested in
    n_cycles=300, 
    dt=0.001,
)

# Run batch simulations with the samples generated in Cell 1
results = simulator.run_batch_simulations(sample_df)

# Convert results to DataFrame for analysis
results_df = pd.DataFrame(results)


In [ ]:
print("Output names:", simulator.output_names)
results_df

test your simulator with our test function to make sure it ios compatible wih AutoEmulate pipelien 

In [ ]:
# need a test for the simulator 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import autoemulate as ae
from tqdm import tqdm
import os

from autoemulate.experimental_design import LatinHypercube
from autoemulate.compare import AutoEmulate
from autoemulate.plotting import _predict_with_optional_std


preprocessing_methods = [{"name" : "PCA", "params" : {"reduced_dim": 2}}]
em = AutoEmulate()
em.setup(sample_df, results, models=["gp"], scale_output = True, reduce_dim_output=True, preprocessing_methods=preprocessing_methods)


In [ ]:
best_model = em.compare()

In [ ]:
em.summarise_cv()

In [ ]:
#em.plot_eval(model=best_model)
best_model

In [ ]:
## 3) Evaluate the emulator (on the test set)
gp = em.get_model('GaussianProcess')
em.evaluate(gp)

In [ ]:
gp
gp_final = em.refit(gp)
em.plot_eval(gp_final)

In [ ]:
print("Available methods:", [method for method in dir(gp_final) if callable(getattr(gp_final, method))])

now run hjistory matching

In [ ]:
from autoemulate.history_matching import HistoryMatcher
# Define observed data with means and variances
observations = {
    'lv.P_i_min': (0.0, 0.1),   # Minimum of minimum LV pressure
    'lv.P_i_max': (20.0, 0.1),   # Maximum of minimum LV pressure
    'lv.P_i_mean': (15.0, 0.1),  # Mean of minimum LV pressure
    'lv.P_i_range': (15.0, 0.5), # Range of minimum LV pressure
    'lv.P_o_min': (1.0, 0.1),  # Minimum of maximum LV pressure
    'lv.P_o_max': (13.0, 0.1),  # Maximum of maximum LV pressure
    'lv.P_o_mean': (12.0, 0.1), # Mean of maximum LV pressure
    'lv.P_o_range': (20.0, 0.5)  # Range of maximum LV pressure
}
    
# Test generating samples
samples = simulator.sample_inputs(10)

# Create history matcher
hm = HistoryMatcher(
    simulator=simulator,
    observations=observations,  # This needs both means and variances
    threshold=3.0
)

# Run history matching
all_samples, all_impl_scores, emulator = hm.run_history_matching(
    n_waves=20,
    n_samples_per_wave=10,
    use_emulator=True,
    initial_emulator=gp_final,
)

In [ ]:
em.plot_eval(emulator)

In [ ]:
em.evaluate(emulator)

In [ ]:
from autoemulate.history_matching_dashboard import HistoryMatchingDashboard
dashboard = HistoryMatchingDashboard(
    samples=all_samples,
    impl_scores=all_impl_scores,
    param_names=simulator.param_names,  # Use simulator.param_names instead of parameter_names
    output_names=simulator.output_names  # Use simulator.output_names instead of output_names
)
dashboard.display()

In [ ]:
# Extract parameter names and bounds from the dictionary
parameter_names = list(parameters_range.keys())
parameter_bounds = list(parameters_range.values())

# Define the problem dictionary for Sobol sensitivity analysis
problem = {
    'num_vars': len(parameter_names),
    'names': parameter_names,
    'bounds': parameter_bounds
}


In [ ]:
em.sensitivity_analysis(problem=problem)
